In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
email=pd.read_csv(r"D:\projects\Project_2\messages.csv")
email

,subject,message,label
0,job posting - apple-iss research center,content - length : 3386 apple-iss research cen...,0
1,NaN,"lang classification grimes , joseph e . and ba...",0
2,query : letter frequencies for text identifica...,i am posting this inquiry for sergei atamas ( ...,0
3,risk,a colleague and i are researching the differin...,0
4,request book information,earlier this morning i was on the phone with a...,0
...,...,...,...
2888,love your profile - ysuolvpv,hello thanks for stopping by ! ! we have taken...,1
2889,you have been asked to join kiddin,"the list owner of : "" kiddin "" has invited you...",1
2890,anglicization of composers ' names,"judging from the return post , i must have sou...",0
2891,"re : 6 . 797 , comparative method : n - ary co...",gotcha ! there are two separate fallacies in t...,0


In [3]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import re
import nltk
from nltk.corpus import stopwords
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier

In [4]:
print(email.shape)

(2893, 3)


In [5]:
stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

def punct_pc(message):
    punct_count = sum([1 for char in message if char in string.punctuation])
    return (punct_count/(len(message) - message.count(' ')))*100

email['message_length'] = email['message'].apply(lambda x : len(x)-x.count(' '))
email['Punct_pc'] = email['message'].apply(lambda x : punct_pc(x))

def clean_data(message):
    punct = "".join([word.lower() for word in message if word not in string.punctuation])
    splt = re.split('\W+',punct)
    msg = [ps.stem(word) for word in splt if word not in stopwords]
    return msg

In [6]:
X_train,X_test,Y_train,Y_test = train_test_split(email[['message','message_length','Punct_pc']],email['label'],test_size=0.2,random_state=123)

In [7]:
print(pd.crosstab(Y_train,columns = 'label',normalize=True))
print(pd.crosstab(Y_test,columns = 'label',normalize=True))
X_train.head()

col_0     label
label          
0      0.832325
1      0.167675
col_0     label
label          
0      0.839378
1      0.160622


,message,message_length,Punct_pc
1259,the instituto universitario ortega y gasset of...,1621,5.243677
984,the loll is now available by anonymous ftp to ...,192,10.416667
1667,second call for papers : seventh annual worksh...,1465,5.324232
24,"hello , there is someone who knows where can i...",290,5.862069
1224,final call for papers the machine translation ...,4034,4.833912


In [8]:
Tfidf_Vect = TfidfVectorizer(analyzer=clean_data)
Tfidf_vect_fit = Tfidf_Vect.fit(X_train['message'])

X_train_Tfidf_vect = Tfidf_vect_fit.transform(X_train['message'])
X_test_Tfidf_vect = Tfidf_vect_fit.transform(X_test['message'])

X_train_vect = pd.concat([X_train[['message_length','Punct_pc']].reset_index(drop=True) ,
                         pd.DataFrame(X_train_Tfidf_vect.toarray())],axis=1)


X_test_vect = pd.concat([X_test[['message_length','Punct_pc']].reset_index(drop=True) , 
                        pd.DataFrame(X_test_Tfidf_vect.toarray())],axis=1)
X_train_vect.head()

,message_length,Punct_pc,0,1,2,3,4,5,6,7,...,46644,46645,46646,46647,46648,46649,46650,46651,46652,46653
0,1621,5.243677,0.010279,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,192,10.416667,0.025006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1465,5.324232,0.012406,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,290,5.862069,0.048060,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4034,4.833912,0.005683,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
X_test_vect.head()

,message_length,Punct_pc,0,1,2,3,4,5,6,7,...,46644,46645,46646,46647,46648,46649,46650,46651,46652,46653
0,719,4.867872,0.041358,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,17137,3.833810,0.002440,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2111,9.095216,0.008946,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1385,8.519856,0.013815,0.045243,0.110545,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,349,13.753582,0.029362,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
Count_Vect = CountVectorizer(analyzer=clean_data)
Count_vect_fit = Count_Vect.fit(X_train['message'])

X_train_Count_vect = Count_vect_fit.transform(X_train['message'])
X_test_Count_vect = Count_vect_fit.transform(X_test['message'])

X_train_Count_vect = pd.concat([X_train[['message_length','Punct_pc']].reset_index(drop=True) ,
                         pd.DataFrame(X_train_Count_vect.toarray())],axis=1)


X_test_Count_vect = pd.concat([X_test[['message_length','Punct_pc']].reset_index(drop=True) , 
                        pd.DataFrame(X_test_Count_vect.toarray())],axis=1)
X_train_Count_vect.head()

,message_length,Punct_pc,0,1,2,3,4,5,6,7,...,46644,46645,46646,46647,46648,46649,46650,46651,46652,46653
0,1621,5.243677,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,192,10.416667,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1465,5.324232,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,290,5.862069,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4034,4.833912,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
rf = RandomForestClassifier(random_state=123,n_jobs=3)
param = {'n_estimators' : [10,25,50,100,300], 'max_depth' : [10, 20, 50,100, None],'max_features' : [10,50,'auto']}

In [12]:
grid = GridSearchCV(rf,param,cv=5,n_jobs=3)

rf_grid_fit_1 = grid.fit(X_train_vect, Y_train)
pd.DataFrame(rf_grid_fit_1.cv_results_).sort_values('mean_test_score',ascending=False)[0:10]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_max_features,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
74,13.489084,2.392639,0.409515,0.076098,None,auto,300,"{'max_depth': None, 'max_features': 'auto', 'n...",0.974082,0.976242,0.961123,0.963283,0.976190,0.970184,0.006598,1
59,14.731656,0.215548,0.639099,0.183737,100,auto,300,"{'max_depth': 100, 'max_features': 'auto', 'n_...",0.974082,0.974082,0.958963,0.963283,0.976190,0.969320,0.006874,2
73,7.611166,0.340531,0.449473,0.067962,None,auto,100,"{'max_depth': None, 'max_features': 'auto', 'n...",0.969762,0.971922,0.961123,0.963283,0.978355,0.968889,0.006186,3
43,7.834927,0.282923,0.535622,0.098105,50,auto,100,"{'max_depth': 50, 'max_features': 'auto', 'n_e...",0.969762,0.969762,0.958963,0.967603,0.978355,0.968889,0.006186,4
58,7.547523,0.484946,0.687859,0.117389,100,auto,100,"{'max_depth': 100, 'max_features': 'auto', 'n_...",0.969762,0.974082,0.958963,0.963283,0.978355,0.968889,0.007033,4
44,15.093138,0.922654,0.449893,0.074853,50,auto,300,"{'max_depth': 50, 'max_features': 'auto', 'n_e...",0.967603,0.971922,0.961123,0.963283,0.978355,0.968457,0.006185,6
42,6.561250,0.340138,0.650159,0.139324,50,auto,50,"{'max_depth': 50, 'max_features': 'auto', 'n_e...",0.967603,0.971922,0.958963,0.965443,0.976190,0.968024,0.005845,7
57,6.084020,0.279603,0.434280,0.058794,100,auto,50,"{'max_depth': 100, 'max_features': 'auto', 'n_...",0.967603,0.971922,0.961123,0.956803,0.980519,0.967594,0.008296,8
41,5.464527,0.234465,0.575233,0.145778,50,auto,25,"{'max_depth': 50, 'max_features': 'auto', 'n_e...",0.965443,0.976242,0.952484,0.971922,0.971861,0.967590,0.008303,9
72,5.683152,0.183807,0.479899,0.101041,None,auto,50,"{'max_depth': None, 'max_features': 'auto', 'n...",0.969762,0.967603,0.961123,0.956803,0.978355,0.966729,0.007417,10


In [13]:
RF_results_TFIDF = pd.DataFrame(rf_grid_fit_1.cv_results_).sort_values('mean_test_score',ascending=False)
RF_results_TFIDF.to_csv("C:\\Users\\parul\\RF_Results_TFIDF.csv",header=True)

In [14]:
rf = RandomForestClassifier(random_state=123,n_jobs=3)
param = {'n_estimators' : [10,25,50,100,300], 'max_depth' : [10, 20, 50,100, None],'max_features' : [10,50,'auto']}

grid = GridSearchCV(rf,param,cv=5,n_jobs=3)

rf_grid_fit_2 = grid.fit(X_train_Count_vect, Y_train)
pd.DataFrame(rf_grid_fitp2.cv_results_).sort_values('mean_test_score',ascending=False)[0:10]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_max_features,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
59,15.684783,0.371075,0.537504,0.143832,100,auto,300,"{'max_depth': 100, 'max_features': 'auto', 'n_...",0.978402,0.978402,0.961123,0.974082,0.989177,0.976237,0.009054,1
74,14.241709,2.661961,0.454392,0.121563,None,auto,300,"{'max_depth': None, 'max_features': 'auto', 'n...",0.978402,0.978402,0.958963,0.974082,0.989177,0.975805,0.009787,2
57,5.836632,0.409079,0.521047,0.100169,100,auto,50,"{'max_depth': 100, 'max_features': 'auto', 'n_...",0.974082,0.974082,0.963283,0.976242,0.989177,0.975373,0.008256,3
73,8.141556,0.231293,0.528983,0.059203,None,auto,100,"{'max_depth': None, 'max_features': 'auto', 'n...",0.976242,0.978402,0.958963,0.978402,0.984848,0.975371,0.008697,4
58,7.772513,0.402209,0.485193,0.060614,100,auto,100,"{'max_depth': 100, 'max_features': 'auto', 'n_...",0.976242,0.976242,0.958963,0.978402,0.984848,0.974939,0.008588,5
72,5.878363,0.277878,0.475777,0.091105,None,auto,50,"{'max_depth': None, 'max_features': 'auto', 'n...",0.969762,0.976242,0.961123,0.978402,0.987013,0.974508,0.008674,6
56,4.756288,0.189369,0.473007,0.070432,100,auto,25,"{'max_depth': 100, 'max_features': 'auto', 'n_...",0.974082,0.971922,0.969762,0.965443,0.987013,0.973644,0.007269,7
71,5.077282,0.201759,0.438116,0.087260,None,auto,25,"{'max_depth': None, 'max_features': 'auto', 'n...",0.971922,0.969762,0.969762,0.967603,0.987013,0.973213,0.007034,8
43,7.635935,0.629278,0.570371,0.119083,50,auto,100,"{'max_depth': 50, 'max_features': 'auto', 'n_e...",0.971922,0.967603,0.958963,0.976242,0.987013,0.972349,0.009296,9
44,13.905983,0.729522,0.500895,0.085740,50,auto,300,"{'max_depth': 50, 'max_features': 'auto', 'n_e...",0.969762,0.971922,0.961123,0.969762,0.987013,0.971917,0.008410,10


In [15]:
RF_results_CountVector = pd.DataFrame(rf_grid_fit_2.cv_results_).sort_values('mean_test_score',ascending=False)
RF_results_CountVector.to_csv("C:\\Users\\parul\\RF_Results_CountVector.csv",header=True)

In [16]:
gb = GradientBoostingClassifier(random_state=123)
param = {'n_estimators' : [100,150,250], 'learning_rate' :[0.01,0.1],'max_depth':[3,10],'max_features': [300,'auto',None]}

In [17]:
gb_grid = GridSearchCV(gb,param,cv=5,n_jobs=-1)

gb_grid_fit = gb_grid.fit(X_train_ vect, Y_train)  
pd.DataFrame(gb_grid_fit.cv_results_).sort_values('mean_test_score',ascending=False)[0:10]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_max_features,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
30,366.909635,10.864950,0.658957,0.116220,0.1,10,auto,100,"{'learning_rate': 0.1, 'max_depth': 10, 'max_f...",0.980562,0.976242,0.971922,0.976242,0.989177,0.978829,0.005851,1
33,373.718727,8.246221,0.634855,0.037541,0.1,10,None,100,"{'learning_rate': 0.1, 'max_depth': 10, 'max_f...",0.980562,0.976242,0.971922,0.976242,0.989177,0.978829,0.005851,1
35,367.973679,24.574668,0.766514,0.286614,0.1,10,None,250,"{'learning_rate': 0.1, 'max_depth': 10, 'max_f...",0.980562,0.976242,0.971922,0.976242,0.987013,0.978396,0.005102,3
34,389.573027,17.933927,0.619681,0.065906,0.1,10,None,150,"{'learning_rate': 0.1, 'max_depth': 10, 'max_f...",0.980562,0.976242,0.971922,0.976242,0.987013,0.978396,0.005102,3
32,395.710688,19.219689,0.701690,0.068091,0.1,10,auto,250,"{'learning_rate': 0.1, 'max_depth': 10, 'max_f...",0.980562,0.976242,0.971922,0.976242,0.987013,0.978396,0.005102,3
31,391.350767,18.213957,0.761346,0.230865,0.1,10,auto,150,"{'learning_rate': 0.1, 'max_depth': 10, 'max_f...",0.980562,0.976242,0.971922,0.976242,0.987013,0.978396,0.005102,3
20,6.457701,0.198295,0.547616,0.086192,0.1,3,300,250,"{'learning_rate': 0.1, 'max_depth': 3, 'max_fe...",0.976242,0.982721,0.967603,0.976242,0.984848,0.977531,0.006040,7
25,264.092112,3.706711,0.710188,0.088488,0.1,3,None,150,"{'learning_rate': 0.1, 'max_depth': 3, 'max_fe...",0.982721,0.980562,0.963283,0.969762,0.984848,0.976235,0.008302,8
22,260.180289,3.468791,0.601298,0.106040,0.1,3,auto,150,"{'learning_rate': 0.1, 'max_depth': 3, 'max_fe...",0.982721,0.980562,0.963283,0.969762,0.984848,0.976235,0.008302,8
26,439.606449,5.156234,0.716271,0.064353,0.1,3,None,250,"{'learning_rate': 0.1, 'max_depth': 3, 'max_fe...",0.984881,0.980562,0.965443,0.969762,0.980519,0.976233,0.007351,10


In [18]:
GB_results = pd.DataFrame(gb_grid_fit.cv_results_).sort_values('mean_test_score',ascending=False)
GB_results.to_csv("C:\\Users\\parul\\GB_Results.csv",header=True)